Connected to py39 (Python 3.9.19)

In [1]:
import torch
import numpy as np

In [2]:
import sys
import os

# Add the current file's directory to the Python path
current_dir = os.path.dirname(os.path.abspath('..'))
if current_dir not in sys.path:
    sys.path.append(current_dir)

from solution import *

In [3]:
e5m2 = {
    "0":                  0b0_00000_00,
    "smallest_subnormal": 0b0_00000_01,
    "largest_subnormal":  0b0_00000_11,
    "smallest_normal":    0b0_00001_00,
    "largest_normal":     0b0_11110_11,
    "inf":                0b0_11111_00,
    "nan1":               0b0_11111_01,
    "nan2":               0b0_11111_10,
    "nan3":               0b0_11111_11,
    "-0":                 0b1_00000_00,
    "-smallest_subnormal":0b1_00000_01,
    "-largest_subnormal": 0b1_00000_11,
    "-smallest_normal":   0b1_00001_00,
    "-largest_normal":    0b1_11110_11,
    "-inf":               0b1_11111_00,
    "nan4":               0b1_11111_01,
    "nan5":               0b1_11111_10,
    "nan6":               0b1_11111_11,
}

e4m3 = {
    "0":                  0b0_0000_000,
    "smallest_subnormal": 0b0_0000_001,
    "largest_subnormal":  0b0_0000_111,
    "smallest_normal":    0b0_0001_000,
    "largest_normal":     0b0_1110_111,
    "largest_normal_ext": 0b0_1111_110,
    "nan1":               0b0_1111_111,
    "-0":                 0b1_0000_000,
    "-smallest_subnormal":0b1_0000_001,
    "-largest_subnormal": 0b1_0000_111,
    "-smallest_normal":   0b1_0001_000,
    "-largest_normal":    0b1_1110_111,
    "-largest_normal_ext":0b1_1111_110,
    "nan2":               0b1_1111_111,
}

nan_values = {
    2: {
        e5m2['nan1'],
        e5m2['nan2'],
        e5m2['nan3'],
        e5m2['nan4'],
        e5m2['nan5'],
        e5m2['nan6'],
    },
    3: {
        e4m3['nan1'],
        e4m3['nan2'],
    }
}
n_mantissa = 2
   

In [15]:
n_mantissa = 2
y = torch.tensor([0.0, 1.4, 0.6], dtype=torch.bfloat16)
x = bfloat16_to_fp8(y, n_mantissa, rounding='trunc')
print(y)
print(x)
x_int = x.to(torch.int32)
s, e, m = decompose_8bit_e5m2(x_int)
print(s, e, m)
y = fp8_to_bfloat16(x, n_mantissa)
print(x, y)

tensor([0.0000, 1.3984, 0.6016], dtype=torch.bfloat16)
tensor([ 0, 61, 56], dtype=torch.uint8)
tensor([0, 0, 0], dtype=torch.int32) tensor([ 0, 15, 14], dtype=torch.int32) tensor([0, 1, 0], dtype=torch.int32)


tensor([0.], dtype=torch.bfloat16)
tensor([0], dtype=torch.uint8)
tensor([0], dtype=torch.int32) tensor([0], dtype=torch.int32) tensor([0], dtype=torch.int32)


In [20]:
 
# Generate input in the range [-1, 1]
# input = torch.rand(2048, 2048, dtype=torch.bfloat16) + 0.5
input = torch.zeros(1, 1, dtype=torch.bfloat16)
round_up_count = 0
round_down_count = 0

for i in range(100):
    fp8 = round_to_fp8_represented_as_int8(input, n_mantissa, None)
    output = undo_int8_fp8(fp8, n_mantissa)
    
    # Compare input and output to count rounding directions
    rounded_up = (output > input).sum().item()
    rounded_down = (output < input).sum().item()
    unchanged = (output == input).sum().item()
    
    round_up_count += rounded_up
    round_down_count += rounded_down

print(f"Total rounds up: {round_up_count}")
print(f"Total rounds down: {round_down_count}")
print(f"Total unchanged: {unchanged}")
print(f"Fraction rounded up: {round_up_count / (round_up_count + round_down_count + unchanged)}")
# mean of input and output
print(f"Mean of input: {input.mean()}")
print(f"Mean of output: {output.mean()}")


    # assert torch.allclose(input.mean(), output.mean(), rtol=1e-02)

Total rounds up: 0
Total rounds down: 0
Total unchanged: 1
Fraction rounded up: 0.0
Mean of input: 0.0
Mean of output: 0.0


In [12]:
output

tensor([[2.]], dtype=torch.bfloat16)

In [21]:
# Construct the full range of bfloat16
y = torch.arange(0, 2**16, dtype=torch.int32)
y = bits_to_bfloat16(y)
print(y.shape, y)

torch.Size([65536]) tensor([0.0000e+00, 9.1835e-41, 1.8367e-40,  ...,        nan,        nan,
               nan], dtype=torch.bfloat16)


In [4]:
z = bfloat16_to_bits(y)
print(z.shape, z)

torch.Size([65536]) tensor([    0,     1,     2,  ..., 65533, 65534, 65535], dtype=torch.int32)


In [23]:
# Construct the full range of uint8
x = torch.arange(0, 2**8, dtype=torch.int32)
s, e, m = decompose_8bit_e5m2(x)
# Generate fp8 e5m2 as floating points
fp8_e5m2 = torch.zeros(256, dtype=torch.float32)

for i in range(256):
    sign = (i & 0b10000000) >> 7
    exponent = (i & 0b01111100) >> 2
    mantissa = i & 0b00000011

    # Handle special cases
    if exponent == 0:
        if mantissa == 0:
            # Zero
            fp8_e5m2[i] = 0.0 if sign == 0 else -0.0
        else:
            # Subnormal numbers
            fp8_e5m2[i] = ((-1)**sign) * (2**-14) * (mantissa / 4)
    elif exponent == 31:
        if mantissa == 0:
            # Infinity
            fp8_e5m2[i] = float('inf') if sign == 0 else float('-inf')
        else:
            # NaN
            fp8_e5m2[i] = float('nan')
    else:
        # Normal numbers
        fp8_e5m2[i] = ((-1)**sign) * (2**(exponent - 15)) * (1 + mantissa / 4)

print("FP8 E5M2 range:")
print(fp8_e5m2)


FP8 E5M2 range:
tensor([ 0.0000e+00,  1.5259e-05,  3.0518e-05,  4.5776e-05,  6.1035e-05,
         7.6294e-05,  9.1553e-05,  1.0681e-04,  1.2207e-04,  1.5259e-04,
         1.8311e-04,  2.1362e-04,  2.4414e-04,  3.0518e-04,  3.6621e-04,
         4.2725e-04,  4.8828e-04,  6.1035e-04,  7.3242e-04,  8.5449e-04,
         9.7656e-04,  1.2207e-03,  1.4648e-03,  1.7090e-03,  1.9531e-03,
         2.4414e-03,  2.9297e-03,  3.4180e-03,  3.9062e-03,  4.8828e-03,
         5.8594e-03,  6.8359e-03,  7.8125e-03,  9.7656e-03,  1.1719e-02,
         1.3672e-02,  1.5625e-02,  1.9531e-02,  2.3438e-02,  2.7344e-02,
         3.1250e-02,  3.9062e-02,  4.6875e-02,  5.4688e-02,  6.2500e-02,
         7.8125e-02,  9.3750e-02,  1.0938e-01,  1.2500e-01,  1.5625e-01,
         1.8750e-01,  2.1875e-01,  2.5000e-01,  3.1250e-01,  3.7500e-01,
         4.3750e-01,  5.0000e-01,  6.2500e-01,  7.5000e-01,  8.7500e-01,
         1.0000e+00,  1.2500e+00,  1.5000e+00,  1.7500e+00,  2.0000e+00,
         2.5000e+00,  3.0000e+00,  

In [6]:
print(*decompose_16bit(z))
print(*decompose_8bit_e4m3(x))
print(*decompose_8bit_e5m2(x))

tensor([0, 0, 0,  ..., 1, 1, 1], dtype=torch.int32) tensor([  0,   0,   0,  ..., 255, 255, 255], dtype=torch.int32) tensor([  0,   1,   2,  ..., 125, 126, 127], dtype=torch.int32)
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 

In [7]:
sign, exponent, mantissa = decompose_16bit(z)
bits = compose_16bit(sign, exponent, mantissa)
print(torch.equal(bits, z))

True


In [8]:
# Test that you can convert bfloat16 to bits and back
y_ = bits_to_bfloat16(
    bfloat16_to_bits(y)
)
comparison = torch.isclose(
        y,
        y_
)
comparison[torch.isnan(y) & torch.isnan(y_)] = True
comparison.all()

tensor(True)

In [9]:
# Test that you can convert bfloat16 to bits, decompose, recompose and convert back
y_bits = bfloat16_to_bits(y)
components = decompose_16bit(y_bits)
y_bits_ = compose_16bit(*components)
y_ = bits_to_bfloat16(y_bits_)

comparison = torch.isclose(
        y,
        y_
)
comparison[torch.isnan(y) & torch.isnan(y_)] = True
comparison.all()

tensor(True)

In [10]:
# Construct infinity
s = torch.tensor([0], dtype=torch.int32)
e = torch.tensor([0xFF], dtype=torch.int32)
m = torch.tensor([0], dtype=torch.int32)
n = bits_to_bfloat16(compose_16bit(s, e, m))
print(n)

tensor([inf], dtype=torch.bfloat16)


In [11]:
# Construct nan
s = torch.tensor([0], dtype=torch.int32)
e = torch.tensor([0xFF], dtype=torch.int32)
m = torch.tensor([1], dtype=torch.int32)
n = bits_to_bfloat16(compose_16bit(s, e, m))
print(n)

tensor([nan], dtype=torch.bfloat16)


In [12]:
# Test bfloat16 to e5m2 and back
t = torch.tensor([0.3125], dtype=torch.bfloat16)
t_bits = bfloat16_to_bits(t)

encoded = encode_as_e5m2_trunc(t_bits)
decoded = decode_from_e5m2(encoded)
t_quantized = bits_to_bfloat16(decoded)
print(t, t_quantized)

tensor([0.3125], dtype=torch.bfloat16) tensor([0.3125], dtype=torch.bfloat16)


In [13]:
# Test rounding up when converting from bfloat16 to e5m2
s = torch.tensor([0], dtype=torch.int32)
e = torch.tensor([0b01111100], dtype=torch.int32)
m = torch.tensor([0b1011111], dtype=torch.int32)  # Changed to 1011111 to test rounding up
t_bits = compose_16bit(s, e, m)
t = bits_to_bfloat16(t_bits)  # This is 0 01111100 1011111 in binary

encoded = encode_as_e5m2_round_up(t_bits)
decoded = decode_from_e5m2(encoded)
t_quantized = bits_to_bfloat16(decoded)

# Construct the rounded up e5m2 value
encoded_e5m2 = encode_as_e5m2_trunc(t_bits)
s, e, m = decompose_8bit_e5m2(encoded_e5m2)

# Manually round up (without worrying about mantissa overflow)
m += 1

# Compose the rounded e5m2 value
rounded_e5m2 = compose_e5m2(s, e, m)

# Convert back to bfloat16 using decode_from_e5m2
decoded_bits = decode_from_e5m2(rounded_e5m2)
t_expected = bits_to_bfloat16(decoded_bits)

print("Original:", t)
print("Quantized:", t_quantized)
print("Rounded up correctly:", torch.isclose(t_quantized, t_expected))

Original: tensor([0.2178], dtype=torch.bfloat16)
Quantized: tensor([0.2188], dtype=torch.bfloat16)
Rounded up correctly: tensor([True])


In [14]:
# Test encode_as_e5m2 and decode_as_e5m2 for all possible e5m2 values

# Generate all possible 8-bit values (0 to 255)
all_e5m2_values = torch.arange(0, 256, dtype=torch.uint8)

# Convert to int32 for compatibility with our functions
all_e5m2_int32 = all_e5m2_values.to(dtype=torch.int32)

# Decode all e5m2 values to bfloat16 bits
decoded_bits = decode_from_e5m2(all_e5m2_int32)

# Convert to bfloat16
bfloat16_values = bits_to_bfloat16(decoded_bits)

# Encode back to e5m2
encoded_e5m2 = encode_as_e5m2_trunc(bfloat16_to_bits(bfloat16_values))

# Check if the encoding-decoding process is reversible
is_reversible = torch.all(encoded_e5m2 == all_e5m2_int32)

print(f"Encoding-decoding process is reversible for all e5m2 values: {is_reversible}")

# Additional checks
if not is_reversible:
    # Find where the mismatch occurs
    mismatch = encoded_e5m2 != all_e5m2_int32
    mismatch_indices = torch.where(mismatch)[0]
    print(f"Mismatches found at indices: {mismatch_indices}")
    
    # Print a few mismatched values for inspection
    for idx in mismatch_indices[:5]:  # Show first 5 mismatches
        original = all_e5m2_int32[idx].item()
        encoded = encoded_e5m2[idx].item()
        print(f"Index {idx}: Original {original:08b}, Encoded {encoded:08b}")



Encoding-decoding process is reversible for all e5m2 values: True


In [29]:
# Test round_to_fp8_represented_as_int8() and undo_int8_fp8() for all bfloat16 values

# Generate all possible 16-bit values (0 to 65535)
all_bfloat16_values = torch.arange(0, 2**16, dtype=torch.int32)

# Convert to bfloat16
bfloat16_values = bits_to_bfloat16(all_bfloat16_values)

# Round to fp8 (e5m2)
fp8_values = round_to_fp8_represented_as_int8(bfloat16_values, n_mantissa=2)

# Convert back to bfloat16
recovered_bfloat16 = undo_int8_fp8(fp8_values, n_mantissa=2)

# Create a mask for non-NaN values
non_nan_mask = ~torch.isnan(bfloat16_values) & ~torch.isnan(recovered_bfloat16)

# Check if the rounding process preserves order (ignoring NaNs)
is_order_preserved = torch.all(torch.argsort(bfloat16_values[non_nan_mask]) == torch.argsort(recovered_bfloat16[non_nan_mask]))

print(f"Rounding process preserves order (ignoring NaNs): {is_order_preserved}")


Rounding process preserves order (ignoring NaNs): False


tensor([ 2.0000,  2.0000,  2.5000,  ..., -0.8750, -0.8750, -1.2500],
       dtype=torch.bfloat16)

At this point we are able to:
- Construct the full range of bfloat16, uint8
- Convert bfloat16 to bits and back
- Convert bfloat16 to sign, exponent, mantissa and back
- Test that conversion is correct except NaN values
- Truncate when bfloat16 to e5m2 with mantissa chopping
- Round up when converting from bfloat16 to e5m2

Next:
- Handle subnormal, inf, nan special cases in encode and decode 
- Round by stochastic rounding scheme